## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts()[application_df["APPLICATION_TYPE"].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts().loc[lambda x: x > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < 100].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         4,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,461 (25.24 KB)

 Trainable params: 6,461 (25.24 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 396us/step - accuracy: 0.7121 - loss: 0.5881
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7289 - loss: 0.5547
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7290 - loss: 0.5524
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - accuracy: 0.7282 - loss: 0.5479
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - accuracy: 0.7347 - loss: 0.5459
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - accuracy: 0.7310 - loss: 0.5462
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step - accuracy: 0.7288 - loss: 0.5489
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.7357 - loss: 0.5412
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - accuracy: 0.7360 - loss: 0.5462
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.7291 - loss: 0.5460
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.7361 - loss: 0.5405
Epoch 12/100
804/80

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 550us/step - accuracy: 0.7293 - loss: 0.5578
Loss: 0.5577963590621948, Accuracy: 0.7293294668197632


In [18]:
#Optimize the model

# STEP 1: Install keras-tuner if needed
!pip install keras-tuner --upgrade

# STEP 2: Imports
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# STEP 3: Define model builder function
def build_model(hp):
    model = Sequential()
    
    # Input layer
    model.add(Dense(
        units=hp.Int('units_input', min_value=32, max_value=256, step=32),
        activation='relu',
        input_dim=X_train_scaled.shape[1]
    ))
    
    # Hidden layers
    for i in range(hp.Int('num_hidden_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
            activation='relu'
        ))
        model.add(Dropout(rate=hp.Choice(f'dropout_{i}', values=[0.1, 0.2, 0.3])))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# STEP 4: Initialize tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='alphabet_soup_tuning'
)

# STEP 5: Run hyperparameter search
stop_early = EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# STEP 6: Retrieve the best model
best_hps = tuner.get_best_hyperparameters(1)[0]
print(f"Best Hyperparameters:\n"
      f"- Input Units: {best_hps.get('units_input')}\n"
      f"- Layers: {best_hps.get('num_hidden_layers')}\n"
      f"- Learning Rate: {best_hps.get('learning_rate')}")

# STEP 7: Train final model
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100)

# STEP 8: Evaluate
loss, accuracy = final_model.evaluate(X_test_scaled, y_test)
print(f"Final Loss: {loss:.4f}, Final Accuracy: {accuracy:.4f}")

Trial 90 Complete [00h 00m 10s]
val_accuracy: 0.7381924390792847

Best val_accuracy So Far: 0.7407191395759583
Total elapsed time: 00h 05m 26s
Best Hyperparameters:
- Input Units: 128
- Layers: 3
- Learning Rate: 0.0001
Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 846us/step - accuracy: 0.5799 - loss: 0.6721 - val_accuracy: 0.7279 - val_loss: 0.5759
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.7148 - loss: 0.5903 - val_accuracy: 0.7328 - val_loss: 0.5562
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.7157 - loss: 0.5789 - val_accuracy: 0.7335 - val_loss: 0.5514
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - accuracy: 0.7222 - loss: 0.5665 - val_accuracy: 0.7345 - val_loss: 0.5495
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.7297 - loss: 0.5643 - val_accuracy: 0.7347 - val_loss: 0.5475
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7244 - loss: 0.5619 - val_accuracy: 0.7366 - val_loss: 0

In [17]:
# Export our model to HDF5 file
nn.save("output/AlphabetSoupCharity_Optimization.h5")